In [5]:
from collections import namedtuple
from pprint import pformat

In [11]:
class Node(namedtuple('Node','location,left_set,right_set')):
    def __repr__(self):
        return pformat(tuple(self))

In [22]:
b=[1,2,3,4,5,6,7]
b[:3]

[1, 2, 3]

In [23]:
b[4:]

[5, 6, 7]

In [31]:
def KDTree(data_set,depth=0):
    if not data_set :
        return None
    data_set.sort(key=lambda x:x[depth])
    split_pos=len(data_set)//2
    median=data_set[split_pos]
    depth=(depth+1) % len(data_set[0])
    
    ## 返回 node实例,左右数据集,继续创建节点
    return Node(location=median,
                left_set=KDTree(data_set[:split_pos],depth),
                right_set=KDTree(data_set[split_pos+1:],depth)
    )
    

In [86]:
data=[(2,3), (5,4), (9,6), (4,7), (8,1), (7,2)]
tree=KDTree(data,0)
print(tree)

((7, 2),
 ((5, 4), ((2, 3), None, None), ((4, 7), None, None)),
 ((9, 6), ((8, 1), None, None), None))


In [48]:
def preorder(node):
    print(node.location)
    if  node.left_set:
        print('------ left---------')
        preorder(node.left_set)
    if  node.right_set:
        print('------ right---------')
        preorder(node.right_set)

In [49]:
preorder(tree)

(7, 2)
------ left---------
(5, 4)
------ left---------
(2, 3)
------ right---------
(4, 7)
------ right---------
(9, 6)
------ left---------
(8, 1)


node2

In [97]:
class Node2:
    def __init__(self,location,split,left_set,right_set):
        self.location=location
        self.split=split
        self.left_set=left_set
        self.right_set=right_set
    
    def __repr__(self):
        return pformat(tuple(self))

In [98]:
def CreateNode2(split,data_set):
    if not data_set:
        return None
    data_set.sort(key=lambda x:x[split])
    split_pos=len(data_set)//2
    median=data_set[split_pos]
    split_next=(split+1) % len(data_set[split])
    print(median)
    ## recursive  create kd node
    return Node2(median,
                 split,
                CreateNode2(split_next,data_set[:split_pos]),
                CreateNode2(split_next,data_set[split_pos+1:]))
    

In [100]:
data2=[(2,3), (5,4), (9,6), (4,7), (8,1), (7,2)]
kdtree2=CreateNode2(0,data2)
# print(kdtree2)

(7, 2)
(5, 4)
(2, 3)
(4, 7)
(9, 6)
(8, 1)


In [72]:
preorder(kdtree2)

(7, 2)
------ left---------
(5, 4)
------ left---------
(2, 3)
------ right---------
(4, 7)
------ right---------
(9, 6)
------ left---------
(8, 1)


node3

In [91]:
class KDNode3(namedtuple('Node','split,location,left,right')):
    def __repr__(self):
        return pformat(tuple(self))

In [92]:
def CreateNode3(split,data_set):
    if not data_set :
        return None
    data_set.sort(key=lambda x:x[split])
    split_pos=len(data_set) //2
    median=data_set[split_pos]
    split_next=(split+1) % len(data_set[0])
    
    return KDNode3(split,
                  median,
                  CreateNode3(split_next,data_set[:split_pos]),
                  CreateNode3(split_next,data_set[split_pos+1:]))
    

In [93]:
kdtree3=CreateNode3(0,data2)
print(kdtree3)

(0,
 (7, 2),
 (1, (5, 4), (0, (2, 3), None, None), (0, (4, 7), None, None)),
 (1, (9, 6), (0, (8, 1), None, None), None))


In [103]:
kdtree3.left

(1, (5, 4), (0, (2, 3), None, None), (0, (4, 7), None, None))

In [131]:
kdtree3.right.left

(0, (8, 1), None, None)

find nearest

In [133]:
print (kdtree3.right.right)
if kdtree3.right.right is None :
    print("is none")
else :
    print("not none")

None
is none


In [134]:
float('inf')>1

True

In [140]:
import numpy as np

In [123]:
result=namedtuple('Result_tuple','nearest_point,nearest_dist,nodes_visited')

In [145]:
def find_nearest(tree,point):
    k=len(point)
    def travel(kd_node,target,max_dist):
        if kd_node is None:
            return result([0] *k,float('inf'),0)
        
        nodes_visited=1
        s=kd_node.split # 进行分割的维度
        pivot=kd_node.location # 进行分割的中心
       ########## 判断目标点再数据集的那个区域,左边或右边 
        if target[s] <=pivot[s]:
            nearer_node=kd_node.left
            further_node=kd_node.right
            
        else :
            nearer_node=kd_node.right
            further_node=kd_node.left
            
        ########递归查找,直到返回为 None
        temp1=travel(nearer_node,target,max_dist)
        
        ##############################找到了
        nearest=temp1.nearest_point
        dist=temp1.nearest_dist
        nodes_visited+=temp1.nodes_visited
        
        if dist<max_dist:   #########max_dist 前面一直是无穷
            max_dist=dist
            
        temp_dist=abs(pivot[s]-target[s])     ######是none前一次的值
        if max_dist<temp_dist:
            return result(nearest,dist,nodes_visited)
        
        ######### tool func #################
        ## 1 computer dist between tart and piovt
        temp_dist=np.sqrt(sum((p1-p2)**2 for p1,p2 in zip(pivot,target)))
        
        if temp_dist<dist:   #######找到上一层,改变初始dist数据
            nearest=pivot
            dist=temp_dist
            max_dist=dist
            
        ## check other point if nearer
        temp2=travel(further_node,target,max_dist)
        
        nodes_visited+=temp2.nodes_visited
        if temp2.nearest_dist<dist:
            nearest=temp2.nearest_point
            distt=temp2.nearest_dist
        
        return result(nearest,dist,nodes_visited)
    
    return travel(tree,point,float('inf'))

In [149]:
kd = CreateNode3(0,data)
ret = find_nearest(kd, [3,4.5])
print(ret)

Result_tuple(nearest_point=(2, 3), nearest_dist=2.0615528128088303, nodes_visited=4)


In [158]:
np.sqrt(np.sum((p1-p2)**2 for p1,p2 in zip([4,7],[2,4.5])))

3.2015621187164243

In [162]:
np.sqrt(np.sum((p1-p2)**2 for p1,p2 in zip([5,4],[2,4.5])))

3.0413812651491097

In [ ]:
# result=namedtuple('Result_tuple','nearest_point,nearest_dist,nodes_visited')

In [245]:
def find_nearest2(tree,target):
    k=len(target)
    def travel(tree,target,radius):
        if tree is None:
            return result([0]*k ,float('inf'),0)
        
        page_visited=1
        page_split=tree.split
        page_point=tree.location
        
        if target[page_split]<page_point[page_split]:
            near_data=tree.left
            far_data=tree.right
        else:
            near_data=tree.right
            far_data=tree.left
        
        temp=travel(near_data,target,radius)
        
        page_visited+=temp.nodes_visited
        
        return_point=temp.nearest_point
        return_dist=temp.nearest_dist
        return_visited=temp.nodes_visited
        
        if return_dist <radius:
            radius=return_dist
            
        test_dist=np.abs(target[page_split]-page_point[page_split])
        
        if test_dist> radius:
            return result(return_point,return_dist,page_visited)
        
        temp_dist=np.sqrt(sum((p1-p2)**2 for p1,p2 in zip(target,page_point)))
        
        if temp_dist < return_dist:
            return_dist=temp_dist
            radius = temp_dist
            return_point=page_point
            
        temp2=travel(far_data,target,radius)
        
        page_visited+=temp2.nodes_visited
        
        if temp2.nearest_dist < return_dist:
            return_dist=temp2.nearest_dist
            return_point=temp2.nearest_point
        
        return result(return_point,return_dist,page_visited)
    return travel(tree,target,float('inf'))

In [246]:
data = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]
kd = CreateNode3(0,data)
find_nearest2(kd,[2,4.5])

Result_tuple(nearest_point=[2, 3], nearest_dist=1.5, nodes_visited=4)

In [242]:
np.sqrt(sum((p1-p2)**2 for p1,p2 in zip([2,4.5],[7,2])))

5.5901699437494745

In [247]:
from time import clock
from random import random

# 产生一个k维随机向量，每维分量值在0~1之间
def random_point(k):
    return [random() for _ in range(k)]
 
# 产生n个k维随机向量 
def random_points(k, n):
    return [random_point(k) for _ in range(n)]    

In [248]:
N = 400000
data4=random_points(3, N)

In [256]:

t0 = clock()
kd2 = CreateNode3(0,data4)           # 构建包含四十万个3维空间样本点的kd树
ret2 = find_nearest2(kd2, [0.1,0.5,0.8])      # 四十万个样本点中寻找离目标最近的点
t1 = clock()
print ("time: ",t1-t0, "s")
print (ret2)

time:  4.50529144713073 s
Result_tuple(nearest_point=[0.10414390617772129, 0.4892394364727126, 0.8075517994494747], nearest_dist=0.013783735370342488, nodes_visited=93)
